In [ ]:
%matplotlib widget

import sys
import os
from pathlib import Path

SCRIPT_DIR = Path(os.getcwd()).parent
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from python.fluorophores import FlStatic, FlCollection
from python.psfs import PsfVectorial
from python.estimators import est_quad2Diter
from python.estimators import est_quadraticdirect1D
from python.simulators import Simulator

# Vectorial PSF

In [ ]:
psf_vec = PsfVectorial()
psf_vec.setpinhole(AU=1)
psf_vec.zerooffset=0.000
sys_aberr = {}
sys_aberr['Zr'] = np.zeros((1,3))
sys_aberr['Zr'][0,0], sys_aberr['Zr'][0,1], sys_aberr['Zr'][0,2] = 4, 0, 0.0  # spherical aberrations 
sys_aberr['maskshift'] = [0,0]
psf_vec.setpar(**sys_aberr)

In [ ]:
fl = FlStatic(brightness=1000)  # define a static fluorophore
fl.pos = [10, 0, 0]

sim = Simulator(fluorophores=fl)

numberOfLocalizations=1000

In [ ]:
# define scan pattern
L = 75
zeroposx = np.atleast_2d(np.array([-1,1,0])*L/2)
probecenter = True  # should we also probe the center?
orbitpoints = 6
laserpowerdonut = 10  # relative, increases brightness
laserpowertophat = 80
laserpowerpf = 8
pointdwelltime = 0.1  # ms, measurement time in each point
repetitions = 1  # how often to repeat the pattern scan
sim.definePattern("donut", psf_vec, 
                  phasemask="vortex",
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowerdonut, 
                  repetitions=repetitions)
                  
sim.definePattern("tophat_xy", psf_vec, 
                  phasemask="tophat",
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowertophat, 
                  repetitions=repetitions,
                  dim=(0,1))

sim.definePattern("pf_x", psf_vec, 
                  phasemask="halfmoonx",
                  zeropos = zeroposx,
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowerpf, 
                  repetitions=repetitions,
                  dim=(0,))

sim.definePattern("pf_y", psf_vec, 
                  phasemask="halfmoony",
                  zeropos = zeroposx,
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowerpf, 
                  repetitions=repetitions,
                  dim=(1,))

In [ ]:
sim.defineComponent("estdonut", "estimator", est_quad2Diter, parameters=[L, probecenter], dim=(0,1))
sim.defineComponent("est_x", "estimator", est_quadraticdirect1D, parameters=[L], dim=(0,))
sim.defineComponent("est_y", "estimator", est_quadraticdirect1D, parameters=[L], dim=(1,))

In [ ]:
seq = ["donut", "estdonut"]
out = sim.runSequence(seq, maxlocs=numberOfLocalizations)
print("no aberration donut:")
sim.summarize_results(out)
xs = np.arange(0, 70, 5)
displaywhat="rmse"
plt.figure()
sim.scan_fov(seq, xs, clearfigure=True, tag="donut", ax1=displaywhat, linestyle='k')

In [ ]:
seqthnoab = ["tophat_xy", "estdonut"]
out = sim.runSequence(seqthnoab, maxlocs=numberOfLocalizations)
print('no aberration tophat:')
sim.summarize_results(out)  # display summary of simulation
sim.scan_fov(seqthnoab, xs, tag="tophat", ax1=displaywhat, linestyle='r')

In [ ]:
seqpf = ["pf_x", "est_x", "pf_y", "est_y"]
out = sim.runSequence(seqpf, maxlocs=numberOfLocalizations)
print('no aberration phaseflux:')
sim.summarize_results(out)  # display summary of simulation
sim.scan_fov(seqpf, xs, tag="phaseflux", ax1=displaywhat, linestyle='b')

In [ ]:
psf_veca = PsfVectorial()
psf_veca.setpinhole(AU=1)
psf_veca.zerooffset=0.000
sys_aberr = {}
sys_aberr['Zr'] = np.zeros((1,3))
sys_aberr['Zr'][0,0], sys_aberr['Zr'][0,1], sys_aberr['Zr'][0,2] = 4, 0, 0.15  # spherical aberrations 
sys_aberr['maskshift'] = [0,0]
psf_veca.setpar(**sys_aberr)

In [ ]:
sim.definePattern("donuta", psf_veca, 
                  phasemask="vortex",
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowerdonut, 
                  repetitions=repetitions)
                  
sim.definePattern("tophat_xya", psf_veca, 
                  phasemask="tophat",
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowertophat, 
                  repetitions=repetitions,
                  dim=(0,1))

sim.definePattern("pf_xa", psf_veca, 
                  phasemask="halfmoonx",
                  zeropos = zeroposx,
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowerpf, 
                  repetitions=repetitions,
                  dim=(0,))

sim.definePattern("pf_ya", psf_veca, 
                  phasemask="halfmoony",
                  zeropos = zeroposx,
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowerpf, 
                  repetitions=repetitions,
                  dim=(1,))

In [ ]:
seqdab = ["donuta", "estdonut"]
out = sim.runSequence(seqdab, maxlocs=numberOfLocalizations)

sim.scan_fov(seqdab, xs, tag="donut ab", ax1=displaywhat, linestyle='k-.')

In [ ]:
seqthab = ["tophat_xya", "estdonut"]
out = sim.runSequence(seqthab, maxlocs=numberOfLocalizations)

sim.scan_fov(seqthab, xs, tag="tophat ab", ax1=displaywhat, linestyle='r-.')

In [ ]:
seqpfa = ["pf_x", "est_x", "pf_y", "est_y"]
out = sim.runSequence(seqpfa, maxlocs=numberOfLocalizations)
sim.scan_fov(seqpfa, xs, tag="phaseflux ab", ax1=displaywhat, linestyle='b-.')

## Multiple fluorophores

In [ ]:
fc = FlCollection()
fc.add(fl)
fl2 = FlStatic()
fl2.pos = [0, 0, 700]
fc.add(fl2)
fl3 = FlStatic()
fl3.pos = [50, 350, 900]
fc.add(fl3)
sim.fluorophores = fc

In [ ]:
seq = ["donut", "estdonut"]
out = sim.runSequence(seq, maxlocs=numberOfLocalizations)
print("fl donut:")
sim.summarize_results(out)
sim.scan_fov(seq, xs, tag="donut fl", ax1=displaywhat, linestyle='k--')

In [ ]:
seqthnoab = ["tophat_xy", "estdonut"]
out = sim.runSequence(seqthnoab, maxlocs=numberOfLocalizations)
print('fl tophat:')
sim.summarize_results(out)  # display summary of simulation
sim.scan_fov(seqthnoab, xs, tag="tophat fl", ax1=displaywhat, linestyle='r--')

In [ ]:
out = sim.runSequence(seqpf, maxlocs=numberOfLocalizations)
print('fl phaseflux:')
sim.summarize_results(out)  # display summary of simulation
sim.scan_fov(seqpf, xs, tag="phaseflux fl", ax1=displaywhat, linestyle='b--')

## Background

In [ ]:
sim.fluorophores = fl
sim.background = 3
seq = ["donut", "estdonut"]
out = sim.runSequence(seq, maxlocs=numberOfLocalizations)
print("bg donut:")
sim.summarize_results(out)
print(np.mean(out.bg_photons_gt))
sim.scan_fov(seq, xs, tag="bg donut:", ax1=displaywhat, linestyle='k:')

In [ ]:
seqthnoab = ["tophat_xy", "estdonut"]
out = sim.runSequence(seqthnoab, maxlocs=numberOfLocalizations)
print('bg tophat:')
sim.summarize_results(out)  # display summary of simulation
print(np.mean(out.bg_photons_gt))
sim.scan_fov(seqthnoab, xs, tag="bg tophat:", ax1=displaywhat, linestyle='r:')

In [ ]:
out = sim.runSequence(seqpf, maxlocs=numberOfLocalizations)
print('bg phaseflux:')
sim.summarize_results(out)  # display summary of simulation
print(np.mean(out.bg_photons_gt))
sim.scan_fov(seqpf, xs, tag="phaseflux bg", ax1=displaywhat, linestyle='b:')

In [ ]:
plt.gca().set_ylim(0, L/2)